In [2]:
import dask.dataframe as dd
import boto3
import os 
s3 = boto3.resource('s3', aws_access_key_id = os.environ['ACCESS'], aws_secret_access_key=os.environ['SECRET'])
my_bucket = s3.Bucket('sar-store')


paths = ["s3://" + object.bucket_name + '/' + object.key for object in my_bucket.objects.filter(Prefix='plates/')]
paths[1:]

['s3://sar-store/plates/xaa.csv',
 's3://sar-store/plates/xab1.csv',
 's3://sar-store/plates/xac2.csv',
 's3://sar-store/plates/xad.csv',
 's3://sar-store/plates/xae.csv',
 's3://sar-store/plates/xaf.csv',
 's3://sar-store/plates/xag.csv',
 's3://sar-store/plates/xah.csv',
 's3://sar-store/plates/xai.csv',
 's3://sar-store/plates/xaj.csv',
 's3://sar-store/plates/xak.csv',
 's3://sar-store/plates/xal.csv',
 's3://sar-store/plates/xam.csv',
 's3://sar-store/plates/xan.csv',
 's3://sar-store/plates/xao.csv',
 's3://sar-store/plates/xap.csv',
 's3://sar-store/plates/xaq.csv',
 's3://sar-store/plates/xar.csv',
 's3://sar-store/plates/xas.csv',
 's3://sar-store/plates/xat.csv',
 's3://sar-store/plates/xau.csv',
 's3://sar-store/plates/xav.csv',
 's3://sar-store/plates/xaw.csv',
 's3://sar-store/plates/xax.csv',
 's3://sar-store/plates/xay.csv',
 's3://sar-store/plates/xaz.csv',
 's3://sar-store/plates/xba.csv',
 's3://sar-store/plates/xbb.csv',
 's3://sar-store/plates/xbc.csv',
 's3://sar-s

In [5]:
df = dd.read_csv(paths[1:], storage_options={'anon': True})

In [6]:
df.drop_duplicates(inplace=True)

,Plate,State,License Type,Summons Number,Issue Date,Violation Time,Violation,Judgment Entry Date,Fine Amount,Penalty Amount,Interest Amount,Reduction Amount,Payment Amount,Amount Due,Precinct,County,Issuing Agency,Violation Status,Summons Image
npartitions=1,,,,,,,,,,,,,,,,,,,
,object,object,object,int64,object,object,object,object,float64,float64,float64,float64,float64,float64,float64,object,object,object,object
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


In [7]:
len(df)

/opt/conda/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)


28672056

In [8]:
types = df.groupby(['License Type'])['Summons Number'].count().compute()

/opt/conda/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)


In [9]:
types.sort_values(ascending=False).head(10)

License Type
PAS    22044751
COM     3856548
OMT     1226339
OMS      356052
SRF      238724
999      210110
IRP      129519
MOT       76441
APP       72458
TRC       62415
Name: Summons Number, dtype: int64

In [159]:
vanities = df[df['License Type'] == 'SRF'].groupby('Plate')['Summons Number'].count().compute()

/opt/conda/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (3,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)
/opt/conda/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)


In [ ]:
summons_per_plate = df.groupby(['Plate', 'License Type'])['Summons Number'].count().compute()

/opt/conda/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (3,8,9,10,11,12,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)
/opt/conda/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)


In [11]:
PAS = df.loc[(df['License Type'].isin(['PAS', 'SRF'])) & (df.State == 'NY')]

In [12]:
PAS['type_1'] = PAS.Plate.str.match(pat = '^\w{3}\B\d{4}$') # ABC-1234
PAS['type_2'] = PAS.Plate.str.match(pat = '^d{6}$') # 123456


PAS['type_3'] = PAS.Plate.str.match(pat = '^\w{1}\B\d{3}\w{2}$')# A12 3BC
PAS['type_4'] = PAS.Plate.str.match(pat = '^\w{1}\B\d{1}\w{1}\d{3}$') # A1B 234
PAS['type_5'] = PAS.Plate.str.match(pat = '^\w{2}\B\d{3}\w{1}$')# AB1 23C
PAS['type_6'] =  PAS.Plate.str.match(pat = '^\d{4}\B\w{3}$') # 1234-ABC
PAS['type_7'] =  PAS.Plate.str.match(pat = '^\w{3}\B\d{3}$') # ABC 123
PAS['type_8'] =  PAS.Plate.str.match(pat = '^\d{1}\w{2}\d{3}$') # 1AB 234
PAS['tlc'] = PAS.Plate.str.match(pat = '^\w{1}\d{6}\w{1}$')  #T702680C


PAS['Standard_Plate'] = PAS.type_1 | PAS.type_2 | PAS.type_3 | PAS.type_4 | PAS.type_5 | PAS.type_6 | PAS.type_7 | PAS.type_8 | PAS.tlc

In [13]:
plate_types = PAS.groupby(['Plate','Standard_Plate'])['Summons Number'].count().compute()

/opt/conda/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)


In [17]:
plate_types.reset_index().groupby('Standard_Plate').mean()

,Summons Number
Standard_Plate,
False,2.938382
True,4.428135


In [27]:
PAS['year'] = PAS['Issue Date'].str.slice(start=6, stop = 10)

In [ ]:
PAS.groupby(['Standard_Plate', 'year'])['Summons Number'].count().compute()

/opt/conda/lib/python3.6/site-packages/dask/local.py:290: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  result = _execute_task(task, data)
